# Langfuse Data Analysis

In [69]:
import json
import os
from datetime import datetime, timedelta
from typing import Optional

from dotenv import load_dotenv
from itables import init_notebook_mode, show
from langfuse import Langfuse
from langfuse.api.resources.commons.types.observations_view import ObservationsView
from pydantic import BaseModel
import pandas as pd
import plotly.express as px

### Utilities Functions

In [70]:
def init_langfuse_client(public_key: str, secret_key: str, host: str):
    return Langfuse(
        public_key=public_key,
        secret_key=secret_key,
        host=host,
    )

In [6]:
def get_all_traces(client: Langfuse, name: Optional[str]=None, from_timestamp: Optional[datetime]=None, to_timestamp: Optional[datetime]=None):
    traces = []
    page = 1

    while True:
        data = client.fetch_traces(name=name, page=page, from_timestamp=from_timestamp, to_timestamp=to_timestamp).data
        if len(data) == 0:
            break
        traces += data
        page += 1

    return traces

In [71]:
def get_all_observations(
    client: Langfuse, 
    name: Optional[str]=None, 
    from_timestamp: Optional[datetime]=None, 
    to_timestamp: Optional[datetime]=None
) -> list[ObservationsView]:
    observations = []
    page = 1

    while True:
        data = client.fetch_observations(name=name, page=page, from_start_time=from_timestamp, to_start_time=to_timestamp).data
        if len(data) == 0:
            break
        observations += data
        page += 1

    return observations

In [8]:
def pprint_json(data):
    print(json.dumps(json.loads(data), indent=2, ensure_ascii=False))

In [9]:
def get_error_results_by_query(error_results, query):
    results = []
    for error_type in error_results.keys():
        results += list(
            filter(
                lambda error_result: error_result.dict()['input']['args'][0]['query'] == query,
                error_results[error_type]
            )
        )
    return results

In [10]:
def get_error_result_details(error_results):
    error_results_details = {}
    for result in error_results:
        error_type = result.output['metadata']['error_type']
        if error_type not in error_results_details:
            error_results_details[error_type] = [result]
        else:
            error_results_details[error_type].append(result)
    return error_results_details

In [11]:
def get_traces_by_conditions(traces, conditions):
    def get_traces_with_some_conditions(traces, conditions):
        results = []
        for trace in traces:
            match = True
            for key, value in conditions.items():
                if key == "metadata":
                    for meta_key, meta_value in value.items():
                        if trace.metadata.get(meta_key) != meta_value:
                            match = False
                            break
                elif getattr(trace, key, None) != value:
                    match = False
                    break
            if match:
                results.append(trace)
        return results
    
    def get_trace_results_by_type(traces):
        error_results = []
        no_error_results = []
        for trace in traces:
            if trace.metadata.get('error_type', ''):
                error_results.append(trace)
            else:
                no_error_results.append(trace)

        assert len(error_results) + len(no_error_results) == len(traces)

        return error_results, no_error_results

    _traces = get_traces_with_some_conditions(traces, conditions)
    print(f'number of traces: {len(_traces)}')

    error_results, no_error_results = get_trace_results_by_type(_traces)
    print(f'# of error results: {len(error_results)}')
    print(f'# of no error results: {len(no_error_results)}')
    print(f'ratio of failed traces: {len(error_results) / len(_traces)}')

    return error_results, no_error_results

In [12]:
def get_traces_group_by_value(traces, name, value):
    results = {}
    for trace in traces:
        if trace.name == name:
            if _val := trace.metadata.get(value, ''):
                if _val not in results:
                    results[_val] = [trace]
                else:
                    results[_val].append(trace)

    return results

## Analysis Code

In [ ]:
init_notebook_mode(all_interactive=True)
load_dotenv(".env", override=True)

client = init_langfuse_client(
    os.getenv("LANGFUSE_PUBLIC_KEY"),
    os.getenv("LANGFUSE_SECRET_KEY"),
    os.getenv("LANGFUSE_HOST")
)

Get all traces

In [ ]:
traces = get_all_traces(client)
len(traces)

Get all Intent Classification observations

In [100]:
class IntentClassificationData(BaseModel):
    class Input(BaseModel):
        query: str
        project_id: Optional[str] = None
        history: Optional[dict] = None


    class Output(BaseModel):
        intent: str
        db_schemas: list[str]

    id: str
    project_id: str
    trace_id: str
    start_time: datetime
    end_time: datetime
    input: Input
    output: Output

In [ ]:
observations = get_all_observations(client, name="Intent Classification")
len(observations)

In [75]:
def group_observations_by_intent(observations: list[IntentClassificationData]):
    intents = set([observation.output.intent for observation in observations])
    results = {}
    for intent in intents:
        results[intent] = [observation for observation in observations if observation.output.intent == intent]

    return results

In [101]:
observation_results = []
for observation in observations:
    observation_results.append(IntentClassificationData(
        trace_id=observation.trace_id,
        start_time=observation.start_time,
        end_time=observation.end_time,
        project_id=observation.projectId,
        id=observation.id,
        input=IntentClassificationData.Input(
            query=observation.input['kwargs']['query'],
            project_id=observation.input['kwargs']['id'],
            history=observation.input['kwargs']['history'],
        ),
        output=IntentClassificationData.Output(
            intent=observation.output['post_process']['intent'],
            db_schemas=observation.output['post_process']['db_schemas'],
        ),
    ))

In [ ]:
pprint_json(observation_results[0].model_dump_json())

In [ ]:

for intent, observations in group_observations_by_intent(observation_results).items():
    print(f'intent: {intent}')
    print(f'number of observations: {len(observations)}')

In [ ]:
for _, observations in group_observations_by_intent(observation_results).items():
    for observation in observations:
        query = observation.input.query
        if observation.input.history:
            summaries = [step['summary'] for step in observation.input.history.get('steps', []) if step.get('summary', '')]
            query = ' '.join(summaries) + ' ' + observation.input.query
        print(f'project_id: {observation.input.project_id}')
        print(f'url: {os.getenv('LANGFUSE_HOST')}/project/{observation.project_id}/traces/{observation.trace_id}?observation={observation.id}')
        print(f'query: {query}')
        print(f'intent: {observation.output.intent}')
        print('-' * 100)


Trace names

In [ ]:
set(trace.name for trace in traces)

In [ ]:
RELEASE = "0.8.17"

### Traces: Prepare Semantics

In [20]:
def show_mdl_table_distribution(traces):
    results = []
    for result in traces:
        if isinstance(result.input, dict):
            results.append(
                len(json.loads(result.input['args'][0]['mdl'])['models'])
            )

    df = pd.DataFrame(results, columns=['table_num'])
    fig = px.histogram(df, x="table_num")
    fig.show()

In [21]:
def show_mdl_columns_per_table_distribution(traces):
    results = []
    for result in traces:
        if isinstance(result.input, dict):
            mdl = json.loads(result.input['args'][0]['mdl'])
            for model in mdl['models']:
                results.append(len(model['columns']))

    df = pd.DataFrame(results, columns=['column_num'])
    fig = px.histogram(df, x="column_num")
    fig.show()

In [ ]:
conditions = {
    "metadata": {
    },
    "name": "Prepare Semantics",
    # "release": RELEASE,
}
error_results, no_error_results = get_traces_by_conditions(traces, conditions)

In [ ]:
set([model.metadata['mdl_hash'] for model in error_results + no_error_results])

In [ ]:
show_mdl_table_distribution(error_results + no_error_results)

In [ ]:
show_mdl_columns_per_table_distribution(error_results + no_error_results)

In [ ]:
error_results_details = get_error_result_details(error_results)

for key, value in error_results_details.items():
    print(key)
    print(len(value))

### Traces: Ask Question

In [34]:
def get_dry_run_failed_cases(client, from_timestamp = None, to_timestamp = None):
    results = []
    for data in get_all_observations(client, name="post_process", from_timestamp=from_timestamp, to_timestamp=to_timestamp):
        if invalid_generation_results := data.output.get('invalid_generation_results', []):
            for result in invalid_generation_results:
                result['trace_id'] = data.trace_id
            results += invalid_generation_results
    
    return results

In [ ]:
conditions = {
    "metadata": {
    },
    "name": "Ask Question",
    # "release": RELEASE,
}
error_results, no_error_results = get_traces_by_conditions(traces, conditions)

get number of ask question traces grouped by mdl_hash

In [ ]:
_traces_by_mdl_hash = get_traces_group_by_value(error_results, "Ask Question", "mdl_hash")
sorted_traces_by_mdl_hash = sorted(_traces_by_mdl_hash.items(), key=lambda x: len(x[1]), reverse=True)

print(f'number of mdl_hash: {len(sorted_traces_by_mdl_hash)}')
for mdl_hash, traces in sorted_traces_by_mdl_hash:
    print(f'mdl_hash: {mdl_hash}')
    print(f'size of traces: {len(traces)}')

show ask question failed reaons groupbed by error_type

In [ ]:
dry_run_failed_cases = get_dry_run_failed_cases(client, from_timestamp=datetime.now() - timedelta(days=14))
len(dry_run_failed_cases)

In [ ]:
show(
    pd.DataFrame(dry_run_failed_cases),
    buttons=["csvHtml5"],
    layout={"top1": "searchPanes"},
    searchPanes={"layout": "columns-3", "cascadePanes": True}
)

In [ ]:
error_results_details = get_error_result_details(error_results)
for key, value in error_results_details.items():
    print(key)
    print(len(value))

In [ ]:
for key in error_results_details.keys():
    print(f'Error Type: {key}')
    for error_result in error_results_details[key]:
        pprint_json(error_result.json())

In [ ]:
_error_results = get_error_results_by_query(error_results_details, '我在台中公園，有哪些路線我可以搭乘？')
len(_error_results)

In [ ]:
for _error_result in _error_results:
    pprint_json(_error_result.json())

### Traces: Ask Details(Breakdown SQL)

In [ ]:
conditions = {
    "metadata": {
    },
    "name": "Ask Details(Breakdown SQL)",
    "release": RELEASE,
}
error_results, no_error_results = get_traces_by_conditions(traces, conditions)